This notebook was inpired by [this LlamaIndex example](https://docs.llamaindex.ai/en/stable/examples/finetuning/embeddings/finetune_embedding_adapter/)

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [4]:
# %pip install llama-index-finetuning
# %pip install llama-index-finetuning-callbacks
# %pip install llama-index-llms-openai
# %pip install llama-index pypdf sentence-transformers ragas
%pip install spacy

  Using cached spacy-3.7.4-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.10-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (2.0 kB)
  Using cached cymem-2.0.8-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (8.4 kB)
  Using cached preshed-3.0.9-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (2.2 kB)
  Using cached thinc-8.2.3-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (15 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.4.8-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.3.4-py3-none-any.whl.metadata (4.7 kB)
  Using cached typer-0.9.

In [1]:
from bubls.utils.data.download import download_file_from_url
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import DatasetGenerator
import random
# from bubls.utils.data.load import load_corpus
# from bubls.utils.evaluation.evaluate_embeddings import (
#     get_query_hit_pairs, sentence_transformer_ir_evaluator
# )
# from bubls.utils.custom_models.embedding_adapter import CustomNN
# from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
# from llama_index.finetuning import generate_qa_embedding_pairs
# from llama_index.llms.openai import OpenAI
# from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
# from llama_index.core.embeddings import resolve_embed_model
# import torch
import os

# # from llama_index.finetuning import SentenceTransformersFinetuneEngine
# from llama_index.embeddings.openai import OpenAIEmbedding
# from llama_index.embeddings.adapter.utils import TwoLayerNN
# from llama_index.embeddings.adapter import LinearAdapterEmbeddingModel

<jemalloc>: Unsupported system page size


## Defining Global Variables

In [2]:
METADATA = {
    "train": {
        "ipcc_ch3": {
            "source_url": "https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf",
            "file_name": "IPCC_AR6_WGII_Chapter03.pdf",
            "save_data_to": os.path.join(os.environ["DATA_DIR"], "ipcc_ch3"),
        }
    },
    # "val": {
    #     "uber_10k": {
    #         "source_url": "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf",
    #         "file_name": "uber_10k_2021.pdf",
    #         "save_data_to": os.path.join(os.environ["DATA_DIR"], "uber_10k"),
    #     }
    # }
}

PERSIST_FINETUNE_DATA_TO = os.path.join(os.environ["PERSIST_DIR"], "eg1_finetune_llm")

QUESTION_GEN_QUERY = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context, formulate "
    "a single question that captures an important fact from the "
    "context. Restrict the question to the context information provided."
)


## Ingest Data
- Download Information
- Use simple directory reader to load data
- Use DatasetGenerator and gpt3.5 to generate a dataset based on a query request.

In [3]:
data = {}
documents = {}
for split in METADATA:
    files = []
    for k, md in METADATA[split].items():
        files.append(
            download_file_from_url(md["source_url"], md["file_name"], md["save_data_to"])
        )
    data_path = os.path.join(PERSIST_FINETUNE_DATA_TO, f"{split}_data.txt")
    if not os.path.exists(data_path):
        documents[split] = SimpleDirectoryReader(input_files=files).load_data()
        random.shuffle(documents[split])

        dataset_generator = DatasetGenerator.from_documents(
            documents[split][:50],
            question_gen_query=QUESTION_GEN_QUERY,
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
        )

        questions = dataset_generator.generate_questions_from_nodes(num=2)
        with open(data_path, "w") as f:
            for question in questions:
                f.write(question + "\n")
            


ImportError: Spacy is not installed, please install it with `pip install spacy`.